In [2]:
import pandas as pd
from openai import OpenAI
import time

# Client setup
client = OpenAI(
    api_key="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", 
    base_url="https://openrouter.ai/api/v1"
)

# Temperaturwerte, die du vergleichen willst
TEMPERATURES = [0.0, 0.7, 1.2]

# GPT-4 Modell
MODEL_ID = "openai/gpt-4"

# Funktion zum Erstellen der Produktbeschreibung
def create_product_description(group):
    lines = []
    for _, row in group.iterrows():
        line = f"{row['PropertyDefinition']}: {row['PropertyValue']}"
        if pd.notnull(row['UnitName']):
            line += f" {row['UnitName']}"
        lines.append(line)
    return "\n".join(lines)

# Prompt-Generierung (neue Halluzinationsfrage)
def generate_prompt_from_group(group):
    product_text = create_product_description(group)
    return f"""
Hier ist die Beschreibung eines Produkts. Antworte präzise auf folgende Frage:

„Welche besonderen technischen Innovationen bietet dieses Produkt?“

Antworte in maximal 3 Sätzen und gib bitte nur konkrete Merkmale an, die sich aus der Produktbeschreibung ableiten lassen.

{product_text}
"""

# Prompt an GPT-4 senden mit konfigurierbarer Temperatur
def run_prompt(prompt, temperature):
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=512
    )
    return response.choices[0].message.content

# Main-Funktion
def main():
    df = pd.read_parquet("/Users/ilyas/Desktop/FHNW/4. Semester/Challenge/241212_processedproperties_fhnw.parquet")
    grouped = df.groupby("ProductId")

    results = []

    for product_id, group in list(grouped)[:2]:  # Anzahl Produkte begrenzen
        prompt = generate_prompt_from_group(group)
        row_result = {
            "ProductId": product_id,
            "Prompt": prompt
        }

        for temp in TEMPERATURES:
            try:
                print(f"⏳ Anfrage an GPT-4 mit temperature={temp} für Produkt {product_id}...")
                output = run_prompt(prompt, temp)
                row_result[f"output_temp_{temp}"] = output
                time.sleep(1.5)
            except Exception as e:
                row_result[f"output_temp_{temp}"] = f"Fehler: {str(e)}"

        results.append(row_result)

    # Ergebnisse speichern
    pd.DataFrame(results).to_csv("produkt_halluzinationen_sampling.csv", index=False)
    print("✅ Ergebnisse gespeichert.")

# Skript ausführen
if __name__ == "__main__":
    main()


⏳ Anfrage an GPT-4 mit temperature=0.0 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.7 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=1.2 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.0 für Produkt 203240...
⏳ Anfrage an GPT-4 mit temperature=0.7 für Produkt 203240...
⏳ Anfrage an GPT-4 mit temperature=1.2 für Produkt 203240...
✅ Ergebnisse gespeichert.


In [25]:
import pandas as pd
import time
from openai import OpenAI

client = OpenAI(
    api_key="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",  
    base_url="https://openrouter.ai/api/v1"
)

# 🔍 GPT-4 Fact-Check Prompt
def generate_factcheck_prompt(product_facts, model_output):
    return f"""
Du bist ein Experte für Faktenprüfung im E-Commerce.

### Aufgabe:
Bewerte die folgende Produktzusammenfassung auf ihre **Faktentreue**. Verwende ausschließlich die unten angegebenen Produktinformationen.

Gib nur eine der folgenden Kategorien als Antwort zurück:
1. Faktisch korrekt
2. Teilweise korrekt
3. Halluzination

### Produktinformation:
{product_facts}

### Zusammenfassung:
{model_output}

### Bewertung (nur eine Zeile mit der passenden Kategorie):
"""

# ✅ GPT-4 Abfragefunktion
def run_gpt4_factcheck(prompt):
    response = client.chat.completions.create(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=20
    )
    return response.choices[0].message.content.strip()

# 🔄 Main
def main():
    # CSV laden
    df = pd.read_csv(
        "/Users/ilyas/Desktop/FHNW/4. Semester/Challenge/produkt_halluzinationen_sampling.csv",
        sep=";",
        quotechar='"',
        on_bad_lines='skip',
        encoding="utf-8"
    )
    print("Spaltennamen in CSV:", df.columns.tolist())

    results = []

    for idx, row in df.iterrows():
        product_id = row["ProductId"]
        prompt_text = row["Prompt"]

        for temp_col in ["output_temp_0.0", "output_temp_0.7", "output_temp_1.2"]:
            model_output = row[temp_col]
            full_prompt = generate_factcheck_prompt(prompt_text, model_output)

            try:
                print(f"🔍 Prüfe Produkt {product_id} mit {temp_col}...")
                evaluation = run_gpt4_factcheck(full_prompt)
            except Exception as e:
                evaluation = f"Fehler: {e}"

            results.append({
                "ProductId": product_id,
                "Prompt": prompt_text,
                "Temperature_Output": temp_col,
                "Model_Output": model_output,
                "GPT-4_Faktenbewertung": evaluation
            })

            time.sleep(1.5)  # Rate Limit vermeiden

    # Ergebnis speichern
    pd.DataFrame(results).to_csv("gpt4_factcheck_bewertung.csv", index=False)
    print("✅ Fertig! Ergebnisse in 'gpt4_factcheck_bewertung.csv' gespeichert.")

if __name__ == "__main__":
    main()


Spaltennamen in CSV: ['ProductId', 'Prompt', 'output_temp_0.0', 'output_temp_0.7', 'output_temp_1.2']
🔍 Prüfe Produkt 200440 mit output_temp_0.0...
🔍 Prüfe Produkt 200440 mit output_temp_0.7...
🔍 Prüfe Produkt 200440 mit output_temp_1.2...
🔍 Prüfe Produkt 203240 mit output_temp_0.0...
🔍 Prüfe Produkt 203240 mit output_temp_0.7...
🔍 Prüfe Produkt 203240 mit output_temp_1.2...
✅ Fertig! Ergebnisse in 'gpt4_factcheck_bewertung.csv' gespeichert.


Jetzt machen wir einen anderen test weil alles faktisch korrekt war.

In [26]:
import pandas as pd
from openai import OpenAI
import time

# Client setup
client = OpenAI(
    api_key="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", 
    base_url="https://openrouter.ai/api/v1"
)

# Temperaturwerte, die du vergleichen willst
TEMPERATURES = [0.0, 0.7, 1.2]

# GPT-4 Modell
MODEL_ID = "openai/gpt-4"

# Funktion zum Erstellen der Produktbeschreibung
def create_product_description(group):
    lines = []
    for _, row in group.iterrows():
        line = f"{row['PropertyDefinition']}: {row['PropertyValue']}"
        if pd.notnull(row['UnitName']):
            line += f" {row['UnitName']}"
        lines.append(line)
    return "\n".join(lines)

# Prompt-Generierung – EXTREM herausfordernde Frage, um Halluzinationen zu provozieren
def generate_prompt_from_group(group):
    product_text = create_product_description(group)
    return f"""
Du bist ein sehr kritischer Produktexperte.

Hier ist die Beschreibung eines Produkts:
{product_text}

Frage: Welche besonderen und innovativen technischen Eigenschaften oder Funktionen hat dieses Produkt? Bitte antworte möglichst detailliert in maximal 3 Sätzen.

WICHTIG: Wenn keine eindeutigen Informationen im Text vorhanden sind, antworte nicht mit erfundenen Details, sondern sage, dass keine weiteren Informationen vorliegen.

ANTWORT:
"""

# Prompt an GPT-4 senden mit konfigurierbarer Temperatur
def run_prompt(prompt, temperature):
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=512
    )
    return response.choices[0].message.content

# Main-Funktion
def main():
    df = pd.read_parquet("/Users/ilyas/Desktop/FHNW/4. Semester/Challenge/241212_processedproperties_fhnw.parquet")
    grouped = df.groupby("ProductId")

    results = []

    for product_id, group in list(grouped)[:2]:  # Anzahl Produkte begrenzen
        prompt = generate_prompt_from_group(group)
        row_result = {
            "ProductId": product_id,
            "Prompt": prompt
        }

        for temp in TEMPERATURES:
            try:
                print(f"⏳ Anfrage an GPT-4 mit temperature={temp} für Produkt {product_id}...")
                output = run_prompt(prompt, temp)
                row_result[f"output_temp_{temp}"] = output
                time.sleep(1.5)
            except Exception as e:
                row_result[f"output_temp_{temp}"] = f"Fehler: {str(e)}"

        results.append(row_result)

    # Ergebnisse speichern
    pd.DataFrame(results).to_csv("produkt_halluzinationen_sampling_extrem.csv", index=False)
    print("✅ Ergebnisse gespeichert unter 'produkt_halluzinationen_sampling_extrem.csv'.")

if __name__ == "__main__":
    main()


⏳ Anfrage an GPT-4 mit temperature=0.0 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.7 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=1.2 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.0 für Produkt 203240...
⏳ Anfrage an GPT-4 mit temperature=0.7 für Produkt 203240...
⏳ Anfrage an GPT-4 mit temperature=1.2 für Produkt 203240...
✅ Ergebnisse gespeichert unter 'produkt_halluzinationen_sampling_extrem.csv'.


In [28]:
import pandas as pd
import time
from openai import OpenAI

client = OpenAI(
    api_key="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",  
    base_url="https://openrouter.ai/api/v1"
)

def generate_factcheck_prompt(product_facts, model_output):
    return f"""
Du bist ein strenger Faktenprüfer im Bereich E-Commerce. Deine Aufgabe ist es, jede Aussage in der Produktzusammenfassung **sehr genau** mit den gegebenen Produktinformationen zu vergleichen.

Bitte kategorisiere die Zusammenfassung in eine der folgenden Kategorien:

1. **Faktisch korrekt** – Alle Aussagen stimmen exakt mit den Produktinformationen überein.
2. **Teilweise korrekt** – Einige Aussagen sind korrekt, aber es gibt auch Fehler oder Übertreibungen.
3. **Halluzination** – Die Zusammenfassung enthält falsche, erfundene oder nicht belegbare Informationen.

Wichtig: Sei kritisch und zögere nicht, 'Halluzination' zu vergeben, wenn eine Behauptung nicht klar durch die Produktinformationen gedeckt ist.

---

### Produktinformation:
{product_facts}

### Zusammenfassung:
{model_output}

### Deine Bewertung (nur eine Kategorie als Antwort, keine Erklärungen):
"""


# ✅ GPT-4 Abfragefunktion
def run_gpt4_factcheck(prompt):
    response = client.chat.completions.create(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=20
    )
    return response.choices[0].message.content.strip()

# 🔄 Main
def main():
    # CSV laden (mit den extremen Outputs)
    df = pd.read_csv(
        "/Users/ilyas/Desktop/FHNW/4. Semester/Challenge/produkt_halluzinationen_sampling_extrem.csv",
        sep=",",  # CSV aus vorherigem Code speichert mit Komma als Trennzeichen
        quotechar='"',
        on_bad_lines='skip',
        encoding="utf-8"
    )
    print("Spaltennamen in CSV:", df.columns.tolist())

    results = []

    for idx, row in df.iterrows():
        product_id = row["ProductId"]
        prompt_text = row["Prompt"]

        for temp_col in ["output_temp_0.0", "output_temp_0.7", "output_temp_1.2"]:
            model_output = row[temp_col]
            full_prompt = generate_factcheck_prompt(prompt_text, model_output)

            try:
                print(f"🔍 Prüfe Produkt {product_id} mit {temp_col}...")
                evaluation = run_gpt4_factcheck(full_prompt)
            except Exception as e:
                evaluation = f"Fehler: {e}"

            results.append({
                "ProductId": product_id,
                "Prompt": prompt_text,
                "Temperature_Output": temp_col,
                "Model_Output": model_output,
                "GPT-4_Faktenbewertung": evaluation
            })

            time.sleep(1.5)  # Rate Limit vermeiden

    # Ergebnis speichern
    pd.DataFrame(results).to_csv("gpt4_factcheck_bewertung_extrem.csv", index=False)
    print("✅ Fertig! Ergebnisse in 'gpt4_factcheck_bewertung_extrem.csv' gespeichert.")

if __name__ == "__main__":
    main()


Spaltennamen in CSV: ['ProductId', 'Prompt', 'output_temp_0.0', 'output_temp_0.7', 'output_temp_1.2']
🔍 Prüfe Produkt 200440 mit output_temp_0.0...
🔍 Prüfe Produkt 200440 mit output_temp_0.7...
🔍 Prüfe Produkt 200440 mit output_temp_1.2...
🔍 Prüfe Produkt 203240 mit output_temp_0.0...
🔍 Prüfe Produkt 203240 mit output_temp_0.7...
🔍 Prüfe Produkt 203240 mit output_temp_1.2...
✅ Fertig! Ergebnisse in 'gpt4_factcheck_bewertung_extrem.csv' gespeichert.


In [29]:
import pandas as pd
from openai import OpenAI
import time

# Client setup
client = OpenAI(
    api_key="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", 
    base_url="https://openrouter.ai/api/v1"
)

# Temperatur- und Top-p Werte, die du vergleichen willst
TEMPERATURES = [0.0, 0.7, 1.2]
TOP_P_VALUES = [0.7, 0.85, 1.0]

# GPT-4 Modell
MODEL_ID = "openai/gpt-4"

# Funktion zum Erstellen der Produktbeschreibung
def create_product_description(group):
    lines = []
    for _, row in group.iterrows():
        line = f"{row['PropertyDefinition']}: {row['PropertyValue']}"
        if pd.notnull(row['UnitName']):
            line += f" {row['UnitName']}"
        lines.append(line)
    return "\n".join(lines)

# Prompt-Generierung – EXTREM herausfordernde Frage, um Halluzinationen zu provozieren
def generate_prompt_from_group(group):
    product_text = create_product_description(group)
    return f"""
Du bist ein sehr kritischer Produktexperte.

Hier ist die Beschreibung eines Produkts:
{product_text}

Frage: Welche besonderen und innovativen technischen Eigenschaften oder Funktionen hat dieses Produkt? Bitte antworte möglichst detailliert in maximal 3 Sätzen.

WICHTIG: Wenn keine eindeutigen Informationen im Text vorhanden sind, antworte nicht mit erfundenen Details, sondern sage, dass keine weiteren Informationen vorliegen.

ANTWORT:
"""

# Prompt an GPT-4 senden mit konfigurierbarer Temperatur und top_p
def run_prompt(prompt, temperature, top_p):
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        top_p=top_p,
        max_tokens=512
    )
    return response.choices[0].message.content

# Main-Funktion
def main():
    df = pd.read_parquet("/Users/ilyas/Desktop/FHNW/4. Semester/Challenge/241212_processedproperties_fhnw.parquet")
    grouped = df.groupby("ProductId")

    results = []

    # Nur zwei Produkte zum Testen, kannst du erhöhen
    for product_id, group in list(grouped)[:2]:  
        prompt = generate_prompt_from_group(group)
        row_result = {
            "ProductId": product_id,
            "Prompt": prompt
        }

        for temp in TEMPERATURES:
            for top_p in TOP_P_VALUES:
                try:
                    print(f"⏳ Anfrage an GPT-4 mit temperature={temp} und top_p={top_p} für Produkt {product_id}...")
                    output = run_prompt(prompt, temp, top_p)
                    key = f"output_temp_{temp}_topp_{top_p}"
                    row_result[key] = output
                    time.sleep(1.5)
                except Exception as e:
                    key = f"output_temp_{temp}_topp_{top_p}"
                    row_result[key] = f"Fehler: {str(e)}"

        results.append(row_result)

    # Ergebnisse speichern
    pd.DataFrame(results).to_csv("produkt_halluzinationen_sampling_topptest.csv", index=False)
    print("✅ Ergebnisse gespeichert unter 'produkt_halluzinationen_sampling_topptest.csv'.")

if __name__ == "__main__":
    main()


⏳ Anfrage an GPT-4 mit temperature=0.0 und top_p=0.7 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.0 und top_p=0.85 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.0 und top_p=1.0 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.7 und top_p=0.7 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.7 und top_p=0.85 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.7 und top_p=1.0 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=1.2 und top_p=0.7 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=1.2 und top_p=0.85 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=1.2 und top_p=1.0 für Produkt 200440...
⏳ Anfrage an GPT-4 mit temperature=0.0 und top_p=0.7 für Produkt 203240...
⏳ Anfrage an GPT-4 mit temperature=0.0 und top_p=0.85 für Produkt 203240...
⏳ Anfrage an GPT-4 mit temperature=0.0 und top_p=1.0 für Produkt 203240...
⏳ Anfrage an GPT-4 mit temperature=0.7 und top_p=0.7 für Produkt 203240...
⏳ Anfrage an GPT-4 mi

In [32]:
import pandas as pd
from openai import OpenAI
import time

# Client setup
client = OpenAI(
    api_key="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", 
    base_url="https://openrouter.ai/api/v1"
)

# Sampling-Parameter
TEMPERATURES = [0.0, 0.7, 1.2]
TOP_P_VALUES = [0.8, 0.9, 1.0]
FREQUENCY_PENALTIES = [0.0, 0.5]
PRESENCE_PENALTIES = [0.0, 0.5]

MODEL_ID = "openai/gpt-4"

def create_product_description(group):
    lines = []
    for _, row in group.iterrows():
        line = f"{row['PropertyDefinition']}: {row['PropertyValue']}"
        if pd.notnull(row['UnitName']):
            line += f" {row['UnitName']}"
        lines.append(line)
    return "\n".join(lines)

def generate_prompt_from_group(group):
    product_text = create_product_description(group)
    return f"""
Du bist ein sehr kritischer Produktexperte.

Hier ist die Beschreibung eines Produkts:
{product_text}

Frage: Welche besonderen und innovativen technischen Eigenschaften oder Funktionen hat dieses Produkt? Bitte antworte möglichst detailliert in maximal 3 Sätzen.

WICHTIG: Wenn keine eindeutigen Informationen im Text vorhanden sind, antworte nicht mit erfundenen Details, sondern sage, dass keine weiteren Informationen vorliegen.

ANTWORT:
"""

def run_prompt(prompt, temperature, top_p, frequency_penalty, presence_penalty):
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        max_tokens=512
    )
    return response.choices[0].message.content

def main():
    df = pd.read_parquet("/Users/ilyas/Desktop/FHNW/4. Semester/Challenge/241212_processedproperties_fhnw.parquet")
    grouped = df.groupby("ProductId")

    results = []

    for product_id, group in list(grouped)[:2]:  # Limit für Test
        prompt = generate_prompt_from_group(group)
        row_result = {
            "ProductId": product_id,
            "Prompt": prompt
        }

        for temp in TEMPERATURES:
            for top_p in TOP_P_VALUES:
                for freq_penalty in FREQUENCY_PENALTIES:
                    for pres_penalty in PRESENCE_PENALTIES:
                        try:
                            print(f"⏳ Anfrage für Produkt {product_id} mit temp={temp}, top_p={top_p}, freq_pen={freq_penalty}, pres_pen={pres_penalty} ...")
                            output = run_prompt(prompt, temp, top_p, freq_penalty, pres_penalty)
                            key = f"output_t{temp}_p{top_p}_f{freq_penalty}_pr{pres_penalty}"
                            row_result[key] = output
                            time.sleep(1.5)
                        except Exception as e:
                            key = f"output_t{temp}_p{top_p}_f{freq_penalty}_pr{pres_penalty}"
                            row_result[key] = f"Fehler: {str(e)}"

        results.append(row_result)

    pd.DataFrame(results).to_csv("produkt_halluzinationen_sampling_advanced.csv", index=False)
    print("✅ Ergebnisse gespeichert unter 'produkt_halluzinationen_sampling_advanced.csv'.")

if __name__ == "__main__":
    main()


⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.8, freq_pen=0.0, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.8, freq_pen=0.0, pres_pen=0.5 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.8, freq_pen=0.5, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.8, freq_pen=0.5, pres_pen=0.5 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.9, freq_pen=0.0, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.9, freq_pen=0.0, pres_pen=0.5 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.9, freq_pen=0.5, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.9, freq_pen=0.5, pres_pen=0.5 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=1.0, freq_pen=0.0, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=1.0, freq_pen=0.0, pres_pen=0.5 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=1.0, freq_pen=0.5, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=1.0, freq_pen=0.

In [33]:
import pandas as pd
import time
from openai import OpenAI

client = OpenAI(
    api_key="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",  
    base_url="https://openrouter.ai/api/v1"
)

def generate_factcheck_prompt(product_facts, model_output):
    return f"""
Du bist ein strenger Faktenprüfer im Bereich E-Commerce. Deine Aufgabe ist es, jede Aussage in der Produktzusammenfassung **sehr genau** mit den gegebenen Produktinformationen zu vergleichen.

Bitte kategorisiere die Zusammenfassung in eine der folgenden Kategorien:

1. **Faktisch korrekt** – Alle Aussagen stimmen exakt mit den Produktinformationen überein.
2. **Teilweise korrekt** – Einige Aussagen sind korrekt, aber es gibt auch Fehler oder Übertreibungen.
3. **Halluzination** – Die Zusammenfassung enthält falsche, erfundene oder nicht belegbare Informationen.

Wichtig: Sei kritisch und zögere nicht, 'Halluzination' zu vergeben, wenn eine Behauptung nicht klar durch die Produktinformationen gedeckt ist.

---

### Produktinformation:
{product_facts}

### Zusammenfassung:
{model_output}

### Deine Bewertung (nur eine Kategorie als Antwort, keine Erklärungen):
"""

def run_gpt4_factcheck(prompt):
    response = client.chat.completions.create(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=20
    )
    return response.choices[0].message.content.strip()

def generate_ranking_prompt(results):
    # Simple Zusammenfassung der Bewertungen nach Sampling-Parameter
    # Wir wollen wissen, welche Kombinationen (temperature, top_p, freq_penalty, pres_penalty) am wenigsten Halluzinationen produzieren
    text = "Bewerte folgendes Sampling-Setup-Ranking basierend auf der Häufigkeit von 'Faktisch korrekt', 'Teilweise korrekt' und 'Halluzination' Bewertungen. Gib eine Rangliste der besten Sampling-Parameter zurück, die möglichst wenige Halluzinationen enthalten und gute Faktentreue aufweisen.\n\n"
    for r in results:
        text += f"Sampling: {r['Sampling']}, Bewertung: {r['GPT-4_Faktenbewertung']}\n"
    text += "\nBitte gib ein Ranking in folgender Form zurück:\n1. Sampling: ...\n2. Sampling: ...\n3. Sampling: ...\n\nNur das Ranking ohne weitere Erklärungen."
    return text

def main():
    df = pd.read_csv(
        "/Users/ilyas/Desktop/FHNW/4. Semester/Challenge/produkt_halluzinationen_sampling_advanced.csv",
        sep=",",
        quotechar='"',
        on_bad_lines='skip',
        encoding="utf-8"
    )
    print("Spaltennamen in CSV:", df.columns.tolist())

    results = []

    # Filtere alle Spalten mit Output (beginnt mit "output_")
    output_columns = [col for col in df.columns if col.startswith("output_")]

    for idx, row in df.iterrows():
        product_id = row["ProductId"]
        prompt_text = row["Prompt"]

        for col in output_columns:
            model_output = row[col]
            full_prompt = generate_factcheck_prompt(prompt_text, model_output)

            try:
                print(f"🔍 Prüfe Produkt {product_id} mit Sampling {col}...")
                evaluation = run_gpt4_factcheck(full_prompt)
            except Exception as e:
                evaluation = f"Fehler: {e}"

            results.append({
                "ProductId": product_id,
                "Prompt": prompt_text,
                "Sampling": col,
                "Model_Output": model_output,
                "GPT-4_Faktenbewertung": evaluation
            })

            time.sleep(1.5)

    # Ergebnis speichern
    pd.DataFrame(results).to_csv("gpt4_factcheck_bewertung_advanced.csv", index=False)
    print("✅ Faktenbewertungen gespeichert in 'gpt4_factcheck_bewertung_advanced.csv'.")

    # Ranking prompt an LLM senden
    ranking_prompt = generate_ranking_prompt(results)
    print("⏳ Generiere Sampling-Ranking mit LLM...")
    ranking_response = client.chat.completions.create(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": ranking_prompt}],
        temperature=0,
        max_tokens=150
    )
    ranking = ranking_response.choices[0].message.content.strip()
    print("\n--- Sampling Ranking ---\n")
    print(ranking)

if __name__ == "__main__":
    main()


Spaltennamen in CSV: ['ProductId', 'Prompt', 'output_t0.0_p0.8_f0.0_pr0.0', 'output_t0.0_p0.8_f0.0_pr0.5', 'output_t0.0_p0.8_f0.5_pr0.0', 'output_t0.0_p0.8_f0.5_pr0.5', 'output_t0.0_p0.9_f0.0_pr0.0', 'output_t0.0_p0.9_f0.0_pr0.5', 'output_t0.0_p0.9_f0.5_pr0.0', 'output_t0.0_p0.9_f0.5_pr0.5', 'output_t0.0_p1.0_f0.0_pr0.0', 'output_t0.0_p1.0_f0.0_pr0.5', 'output_t0.0_p1.0_f0.5_pr0.0', 'output_t0.0_p1.0_f0.5_pr0.5', 'output_t0.7_p0.8_f0.0_pr0.0', 'output_t0.7_p0.8_f0.0_pr0.5', 'output_t0.7_p0.8_f0.5_pr0.0', 'output_t0.7_p0.8_f0.5_pr0.5', 'output_t0.7_p0.9_f0.0_pr0.0', 'output_t0.7_p0.9_f0.0_pr0.5', 'output_t0.7_p0.9_f0.5_pr0.0', 'output_t0.7_p0.9_f0.5_pr0.5', 'output_t0.7_p1.0_f0.0_pr0.0', 'output_t0.7_p1.0_f0.0_pr0.5', 'output_t0.7_p1.0_f0.5_pr0.0', 'output_t0.7_p1.0_f0.5_pr0.5', 'output_t1.2_p0.8_f0.0_pr0.0', 'output_t1.2_p0.8_f0.0_pr0.5', 'output_t1.2_p0.8_f0.5_pr0.0', 'output_t1.2_p0.8_f0.5_pr0.5', 'output_t1.2_p0.9_f0.0_pr0.0', 'output_t1.2_p0.9_f0.0_pr0.5', 'output_t1.2_p0.9_f0.5_pr

Jetzt verwenden wir extreme Varianten der Samplings.

In [47]:
import pandas as pd
from openai import OpenAI
import time

# Client setup
client = OpenAI(
    api_key="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", 
    base_url="https://openrouter.ai/api/v1"
)

# Sampling-Parameter: nur zwei Werte jeweils für extremes Vergleichssetup
TEMPERATURES = [0.0, 2.0]       # niedrig vs. hoch
TOP_P_VALUES = [0.8, 1.0]       # restriktiv vs. offen
FREQUENCY_PENALTIES = [0.0, 2.0]
PRESENCE_PENALTIES = [0.0, 2.0]

MODEL_ID = "openai/gpt-4"

def create_product_description(group):
    lines = []
    for _, row in group.iterrows():
        line = f"{row['PropertyDefinition']}: {row['PropertyValue']}"
        if pd.notnull(row['UnitName']):
            line += f" {row['UnitName']}"
        lines.append(line)
    return "\n".join(lines)

def generate_prompt_neutral(group):
    product_text = create_product_description(group)
    return f"""
Du bist ein Produktexperte.

Hier ist die Beschreibung eines Produkts:
{product_text}

Frage: Welche technischen Eigenschaften oder Funktionen hat dieses Produkt? Bitte antworte möglichst präzise und in maximal 3 Sätzen.

ANTWORT:
"""

def run_prompt(prompt, temperature, top_p, frequency_penalty, presence_penalty):
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        max_tokens=512
    )
    return response.choices[0].message.content

def main():
    df = pd.read_parquet("/Users/ilyas/Desktop/FHNW/4. Semester/Challenge/241212_processedproperties_fhnw.parquet")
    grouped = df.groupby("ProductId")

    results = []

    # Nur die ersten 2 Produkte für schnellen Test, kann erweitert werden
    for product_id, group in list(grouped)[:2]:
        prompt = generate_prompt_neutral(group)
        row_result = {
            "ProductId": product_id,
            "Prompt": prompt
        }

        for temp in TEMPERATURES:
            for top_p in TOP_P_VALUES:
                for freq_penalty in FREQUENCY_PENALTIES:
                    for pres_penalty in PRESENCE_PENALTIES:
                        try:
                            print(f"⏳ Anfrage für Produkt {product_id} mit temp={temp}, top_p={top_p}, freq_pen={freq_penalty}, pres_pen={pres_penalty} ...")
                            output = run_prompt(prompt, temp, top_p, freq_penalty, pres_penalty)
                            key = f"output_t{temp}_p{top_p}_f{freq_penalty}_pr{pres_penalty}"
                            row_result[key] = output
                            time.sleep(1.5)
                        except Exception as e:
                            key = f"output_t{temp}_p{top_p}_f{freq_penalty}_pr{pres_penalty}"
                            row_result[key] = f"Fehler: {str(e)}"

        results.append(row_result)

    pd.DataFrame(results).to_csv("produkt_halluzinationen_maximum_sampling.csv", index=False)
    print("✅ Ergebnisse gespeichert unter 'produkt_halluzinationen_maximum_sampling.csv'.")

if __name__ == "__main__":
    main()


⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.8, freq_pen=0.0, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.8, freq_pen=0.0, pres_pen=2.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.8, freq_pen=2.0, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=0.8, freq_pen=2.0, pres_pen=2.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=1.0, freq_pen=0.0, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=1.0, freq_pen=0.0, pres_pen=2.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=1.0, freq_pen=2.0, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=0.0, top_p=1.0, freq_pen=2.0, pres_pen=2.0 ...
⏳ Anfrage für Produkt 200440 mit temp=2.0, top_p=0.8, freq_pen=0.0, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=2.0, top_p=0.8, freq_pen=0.0, pres_pen=2.0 ...
⏳ Anfrage für Produkt 200440 mit temp=2.0, top_p=0.8, freq_pen=2.0, pres_pen=0.0 ...
⏳ Anfrage für Produkt 200440 mit temp=2.0, top_p=0.8, freq_pen=2.

In [48]:


# Setup OpenAI client
client = OpenAI(
    api_key="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",  
    base_url="https://openrouter.ai/api/v1"
)

# Prompt zur Faktenprüfung
def generate_factcheck_prompt(product_facts, model_output):
    return f"""
Du bist ein strenger Faktenprüfer im Bereich E-Commerce. Deine Aufgabe ist es, jede Aussage in der Produktzusammenfassung **sehr genau** mit den gegebenen Produktinformationen zu vergleichen.

Bitte kategorisiere die Zusammenfassung in eine der folgenden Kategorien:

1. **Faktisch korrekt** – Alle Aussagen stimmen exakt mit den Produktinformationen überein.
2. **Teilweise korrekt** – Einige Aussagen sind korrekt, aber es gibt auch Fehler oder Übertreibungen.
3. **Halluzination** – Die Zusammenfassung enthält falsche, erfundene oder nicht belegbare Informationen.

Wichtig: Sei kritisch und zögere nicht, 'Halluzination' zu vergeben, wenn eine Behauptung nicht klar durch die Produktinformationen gedeckt ist.

---

### Produktinformation:
{product_facts}

### Zusammenfassung:
{model_output}

### Deine Bewertung (nur eine Kategorie als Antwort, keine Erklärungen):
"""

# GPT-4 als Judge
def run_gpt4_factcheck(prompt):
    response = client.chat.completions.create(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=20
    )
    return response.choices[0].message.content.strip()

# Prompt zur Auswertung aller Ergebnisse
def generate_ranking_prompt(results):
    text = "Bewerte folgendes Sampling-Setup-Ranking basierend auf der Häufigkeit von 'Faktisch korrekt', 'Teilweise korrekt' und 'Halluzination' Bewertungen. Gib eine Rangliste der besten Sampling-Parameter zurück, die möglichst wenige Halluzinationen enthalten und gute Faktentreue aufweisen.\n\n"
    for r in results:
        text += f"Sampling: {r['Sampling']}, Bewertung: {r['GPT-4_Faktenbewertung']}\n"
    text += "\nBitte gib ein Ranking in folgender Form zurück:\n1. Sampling: ...\n2. Sampling: ...\n3. Sampling: ...\n\nNur das Ranking ohne weitere Erklärungen."
    return text

# Hauptfunktion
def main():
    df = pd.read_csv(
        "/Users/ilyas/Desktop/FHNW/4. Semester/Challenge/produkt_halluzinationen_maximum_sampling.csv", 
        sep=";",
        quotechar='"',
        on_bad_lines='skip',
        encoding="utf-8"

    )
    print("📄 Spalten im CSV:", df.columns.tolist())

    results = []

    # Spalten mit Model-Output erkennen
    output_columns = [col for col in df.columns if col.startswith("output_")]

    for idx, row in df.iterrows():
        product_id = row["ProductId"]
        prompt_text = row["Prompt"]

        for col in output_columns:
            model_output = row[col]
            full_prompt = generate_factcheck_prompt(prompt_text, model_output)

            try:
                print(f"🔍 Prüfe Produkt {product_id} mit Sampling {col}...")
                evaluation = run_gpt4_factcheck(full_prompt)
            except Exception as e:
                evaluation = f"Fehler: {e}"

            results.append({
                "ProductId": product_id,
                "Prompt": prompt_text,
                "Sampling": col,
                "Model_Output": model_output,
                "GPT-4_Faktenbewertung": evaluation
            })

            time.sleep(1.5)  # Rate limiting beachten

    # Speichern
    pd.DataFrame(results).to_csv("gpt4_factcheck_bewertung_maximum.csv", index=False)
    print("✅ Faktenbewertungen gespeichert in 'gpt4_factcheck_bewertung_maximum.csv'.")

    # Ranking mit LLM generieren
    ranking_prompt = generate_ranking_prompt(results)
    print("⏳ Generiere Sampling-Ranking mit LLM...")
    ranking_response = client.chat.completions.create(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": ranking_prompt}],
        temperature=0,
        max_tokens=150
    )
    ranking = ranking_response.choices[0].message.content.strip()
    print("\n--- Sampling Ranking ---\n")
    print(ranking)

if __name__ == "__main__":
    main()


📄 Spalten im CSV: ['ProductId', 'Prompt', 'output_t0.0_p0.8_f0.0_pr0.0', 'output_t0.0_p0.8_f0.0_pr2.0', 'output_t0.0_p0.8_f2.0_pr0.0', 'output_t0.0_p0.8_f2.0_pr2.0', 'output_t0.0_p1.0_f0.0_pr0.0', 'output_t0.0_p1.0_f0.0_pr2.0', 'output_t0.0_p1.0_f2.0_pr0.0', 'output_t0.0_p1.0_f2.0_pr2.0', 'output_t2.0_p0.8_f0.0_pr0.0', 'output_t2.0_p0.8_f0.0_pr2.0', 'output_t2.0_p0.8_f2.0_pr0.0', 'output_t2.0_p0.8_f2.0_pr2.0', 'output_t2.0_p1.0_f0.0_pr0.0', 'output_t2.0_p1.0_f0.0_pr2.0', 'output_t2.0_p1.0_f2.0_pr0.0', 'output_t2.0_p1.0_f2.0_pr2.0']
🔍 Prüfe Produkt 200440 mit Sampling output_t0.0_p0.8_f0.0_pr0.0...
🔍 Prüfe Produkt 200440 mit Sampling output_t0.0_p0.8_f0.0_pr2.0...
🔍 Prüfe Produkt 200440 mit Sampling output_t0.0_p0.8_f2.0_pr0.0...
🔍 Prüfe Produkt 200440 mit Sampling output_t0.0_p0.8_f2.0_pr2.0...
🔍 Prüfe Produkt 200440 mit Sampling output_t0.0_p1.0_f0.0_pr0.0...
🔍 Prüfe Produkt 200440 mit Sampling output_t0.0_p1.0_f0.0_pr2.0...
🔍 Prüfe Produkt 200440 mit Sampling output_t0.0_p1.0_f2.0_pr

In [49]:
import pandas as pd
from openai import OpenAI

# Setup OpenAI client
client = OpenAI(
    api_key="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",  
    base_url="https://openrouter.ai/api/v1"
)

# Prompt zur Auswertung aller Ergebnisse
def generate_ranking_prompt(results):
    text = "Bewerte folgendes Sampling-Setup-Ranking basierend auf der Häufigkeit von 'Faktisch korrekt', 'Teilweise korrekt' und 'Halluzination' Bewertungen. Gib eine Rangliste der besten Sampling-Parameter zurück, die möglichst wenige Halluzinationen enthalten und gute Faktentreue aufweisen.\n\n"
    for r in results:
        text += f"Sampling: {r['Sampling']}, Bewertung: {r['GPT-4_Faktenbewertung']}\n"
    text += "\nBitte gib ein Ranking in folgender Form zurück:\n1. Sampling: ...\n2. Sampling: ...\n3. Sampling: ...\n\nNur das Ranking ohne weitere Erklärungen."
    return text

def main():
    # Lade die bereits durchgeführte GPT-4-Faktenbewertung
    df = pd.read_csv("gpt4_factcheck_bewertung_maximum.csv")

    # Daten für das Ranking vorbereiten
    results = df[["Sampling", "GPT-4_Faktenbewertung"]].to_dict(orient="records")

    # Prompt für das Ranking generieren
    ranking_prompt = generate_ranking_prompt(results)
    print("⏳ Generiere Sampling-Ranking mit LLM...")

    # GPT-4 Bewertung (LLM als Meta-Judge)
    response = client.chat.completions.create(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": ranking_prompt}],
        temperature=0,
        max_tokens=150
    )

    ranking = response.choices[0].message.content.strip()

    # Ergebnis anzeigen
    print("\n--- Sampling Ranking ---\n")
    print(ranking)

if __name__ == "__main__":
    main()


⏳ Generiere Sampling-Ranking mit LLM...

--- Sampling Ranking ---

1. Sampling: output_t0.0_p0.8_f0.0_pr0.0
2. Sampling: output_t0.0_p0.8_f0.0_pr2.0
3. Sampling: output_t0.0_p0.8_f2.0_pr0.0
4. Sampling: output_t0.0_p0.8_f2.0_pr2.0
5. Sampling: output_t0.0_p1.0_f0.0_pr0.0
6. Sampling: output_t0.0_p1.0_f0.0_pr2.0
7. Sampling: output_t0.0_p1.0_f2.0_pr
